## Task 1

### Simulation of a simple refrigerator in TESPy

1. Import the necessary packages

In [238]:
from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.connections import Connection, Bus
from CoolProp.CoolProp import PropsSI
import pandas as pd

2. Create the `Network` for the refrigerator and set the units you want to use

In [239]:
refrigerator = Network(T_unit='C', p_unit='bar', h_unit='kJ / kg', m_unit='kg / s')

3. Set the fluids of the system (for easier access in the functions)

In [240]:
wf = "R600a"
wf_dict = {wf: 1}

4. Set the components of the system

In [241]:
val = Valve('throttling-valve')
eva = SimpleHeatExchanger('evaporator')
cond = SimpleHeatExchanger('condenser')
comp = Compressor('compressor')
cc = CycleCloser('cycle-closer')


5. Set the connections of the system and add them to the `Network`

In [242]:
# cXX = Connection(src_comp, 'src_port', dst_comp, 'dst_port', label='XX')
c01 = Connection(eva, 'out1', comp, 'in1', label='1')
c02 = Connection(comp, 'out1', cond, 'in1', label='2')
c02cc = Connection(cond, 'out1', cc, 'in1', label='2cc')
c03 = Connection(cc, 'out1', val, 'in1', label='3')
c04 = Connection(val, 'out1', eva, 'in1', label='4')

refrigerator.add_conns(c01, c02, c02cc, c03, c04)

6. Define the parameters and design variables

In [243]:
# Components
eva.set_attr(Q=150, pr=1)

comp.set_attr(eta_s=0.75)

cond.set_attr(pr=1)

# Connections
c03.set_attr(fluid=wf_dict)

p4_set = PropsSI("P", "Q", 1, "T", 273.15 - 20, wf) / 1e5  # saturation pressure at given temperature
c04.set_attr(p=p4_set)

h1_set = PropsSI("H", "P", p4_set*1e5, "T", 273.15 - 18, wf) / 1e3  # superheated state at given temperature and pressure
c01.set_attr(h=h1_set)

c02.set_attr(T=60)

h2cc_set = PropsSI("H", "Q", 0, "T", 273.15 + 40, wf) / 1e3  # 
c02cc.set_attr(h=h2cc_set)

7. Add busses for heat and power flows

In [244]:
motor = Bus('motor')
motor.add_comps({'comp': comp, 'char': 0.95, 'base': 'bus'})

cooling = Bus('cooling heat')
cooling.add_comps({'comp': eva, 'base': 'bus'})

refrigerator.add_busses(motor, cooling)

8. Run the simulation and calculate the coefficient of performance

In [245]:
refrigerator.solve(mode='design')


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.94e+05   | 2 %        | 1.41e+00   | 3.72e+05   | 2.04e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.55e+04   | 16 %       | 2.23e-02   | 4.37e+05   | 8.20e+03   | 0.00e+00   | 0.00e+00   
 3     | 7.34e+03   | 23 %       | 1.09e-18   | 1.30e+05   | 3.22e+03   | 0.00e+00   | 0.00e+00   
 4     | 3.02e+02   | 39 %       | 0.00e+00   | 5.02e+03   | 1.79e+02   | 0.00e+00   | 0.00e+00   
 5     | 3.83e-01   | 71 %       | 0.00e+00   | 5.53e+00   | 2.66e-01   | 0.00e+00   | 0.00e+00   
 6     | 4.64e-07   | 100 %      | 0.00e+00   | 6.01e-06   | 3.52e-07   | 0.00e+00   | 0.00e+00   
 7     | 1.17e-10   | 100 %      | 0.00e+00   | 7.99e-10   | 1.32e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 7, Calculation time: 0.03 s, Iterations per second: 262.72


9. Extract TESPy results

In [246]:
tespy_results = {
    'Name': ['1', '2', '2cc', '3', '4'],
    'Mass Flow': [c01.m.val, c02.m.val, c02cc.m.val, c03.m.val, c04.m.val],
    'Temperature': [c01.T.val, c02.T.val, c02cc.T.val, c03.T.val, c04.T.val],
    'Pressure': [c01.p.val, c02.p.val, c02cc.p.val, c03.p.val, c04.p.val],
    'Enthalpy': [c01.h.val, c02.h.val, c02cc.h.val, c03.h.val, c04.h.val]
}

tespy_df = pd.DataFrame(tespy_results)
tespy_df

,Name,Mass Flow,Temperature,Pressure,Enthalpy
0,1,0.00064,-18.000000,0.724773,530.612362
1,2,0.00064,60.000000,6.651517,641.757917
2,2cc,0.00064,39.981369,6.651517,296.280073
3,3,0.00064,39.981369,6.651517,296.280073
4,4,0.00064,-20.000000,0.724773,296.280073


### Simulation of a simple refrigerator in Ebsilon Professional

![Ebsilon Simulation](refrigerator.png)

In [252]:
import pandas as pd

# Load the CSV file into a DataFrame
df_material_streams_ebs = pd.read_csv('ebsilon/ebs_material_streams_results.csv')
df_material_streams_ebs

,Unnamed: 0,Name,Mass Flow,Temperature,Pressure,Enthalpy,Entropy,Mass Flow Unit,Temperature Unit,Pressure Unit,Enthalpy Unit,Entropy Unit
0,0,1,0.00064,-17.999983,0.724772,530.612015,2.313165,kg/s,°C,bar,kJ/kg,kJ/kgK
1,1,2,0.00064,60.000008,6.651510,641.757434,2.398282,kg/s,°C,bar,kJ/kg,kJ/kgK
2,2,2cc,0.00064,40.000000,6.651510,296.327148,1.325609,kg/s,°C,bar,kJ/kg,kJ/kgK
3,3,3,0.00064,40.000000,6.651510,296.327148,1.325609,kg/s,°C,bar,kJ/kg,kJ/kgK
4,4,4,0.00064,-20.000000,0.724772,296.327148,1.387733,kg/s,°C,bar,kJ/kg,kJ/kgK


In [248]:
# Select only the relevant columns and rename them
columns_to_select = ['Name', 'Mass Flow', 'Temperature', 'Pressure', 'Enthalpy']
filtered_df_ebs = df_material_streams_ebs[columns_to_select]

filtered_df_ebs

,Name,Mass Flow,Temperature,Pressure,Enthalpy
0,1,0.00064,-17.999983,0.724772,530.612015
1,2,0.00064,60.000008,6.651510,641.757434
2,2cc,0.00064,40.000000,6.651510,296.327148
3,3,0.00064,40.000000,6.651510,296.327148
4,4,0.00064,-20.000000,0.724772,296.327148


In [249]:
# Merge the dataframes on the 'Name' column to compare
comparison_df = pd.merge(filtered_df_ebs, tespy_df, on='Name', suffixes=('_Ebsilon', '_TESPy'))
comparison_df

,Name,Mass Flow_Ebsilon,Temperature_Ebsilon,Pressure_Ebsilon,Enthalpy_Ebsilon,Mass Flow_TESPy,Temperature_TESPy,Pressure_TESPy,Enthalpy_TESPy
0,1,0.00064,-17.999983,0.724772,530.612015,0.00064,-18.000000,0.724773,530.612362
1,2,0.00064,60.000008,6.651510,641.757434,0.00064,60.000000,6.651517,641.757917
2,2cc,0.00064,40.000000,6.651510,296.327148,0.00064,39.981369,6.651517,296.280073
3,3,0.00064,40.000000,6.651510,296.327148,0.00064,39.981369,6.651517,296.280073
4,4,0.00064,-20.000000,0.724772,296.327148,0.00064,-20.000000,0.724773,296.280073


In [250]:
# Calculate relative differences in percentages
comparison_df['Mass Flow Rel Diff (%)'] = (comparison_df['Mass Flow_Ebsilon'] - comparison_df['Mass Flow_TESPy']) / comparison_df['Mass Flow_TESPy'] * 100
comparison_df['Temperature Rel Diff (%)'] = (comparison_df['Temperature_Ebsilon'] - comparison_df['Temperature_TESPy']) / comparison_df['Temperature_TESPy'] * 100
comparison_df['Pressure Rel Diff (%)'] = (comparison_df['Pressure_Ebsilon'] - comparison_df['Pressure_TESPy']) / comparison_df['Pressure_TESPy'] * 100
comparison_df['Enthalpy Rel Diff (%)'] = (comparison_df['Enthalpy_Ebsilon'] - comparison_df['Enthalpy_TESPy']) / comparison_df['Enthalpy_TESPy'] * 100

# Keep only the relative difference columns
rel_diff_columns = ['Name', 'Mass Flow Rel Diff (%)', 'Temperature Rel Diff (%)', 'Pressure Rel Diff (%)', 'Enthalpy Rel Diff (%)']
comparison_df = comparison_df[rel_diff_columns]

# Display the resulting DataFrame
comparison_df.round(3)

,Name,Mass Flow Rel Diff (%),Temperature Rel Diff (%),Pressure Rel Diff (%),Enthalpy Rel Diff (%)
0,1,-0.018,-0.000,-0.0,-0.000
1,2,-0.018,0.000,-0.0,-0.000
2,2cc,-0.018,0.047,-0.0,0.016
3,3,-0.018,0.047,-0.0,0.016
4,4,-0.018,0.000,-0.0,0.016


### Differences between the simulation in TESPy and Ebsilon

- results
- model used
- components
- parameter settings
- ...

#### Positive aspects of TESPy

- easy to use
- 

#### Negative aspects of TESPy

#### Positive aspects of Ebsilon

#### Negative aspects of Ebsilon